In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout,Masking
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import tensorflow as tf
import seaborn as sns
from sklearn.metrics import confusion_matrix
from keras.utils import plot_model

ImportError: cannot import name 'experimental_functions_run_eagerly' from 'tensorflow.python.eager.def_function' (C:\Users\ioann\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tensorflow\python\eager\def_function.py)

In [ ]:
 Load the data
df = pd.read_csv("dt_01.csv")

# Get the target variable
target = df['sale']

# Create a dictionary to store the number of unique values for each categorical variable
unique_values = {}
categorical_vars = ['channel', 'season','period']
for var in categorical_vars:
    unique_values[var] = len(np.unique(df[var]))

# One-hot encode the categorical variables
df = pd.get_dummies(df, columns=categorical_vars)
feature_names = df.columns.tolist()
feature_names = feature_names[3:]
# Get the unique journey IDs
unique_journey_ids = np.unique(df['journey'])

# Create a list to store the sequences and the target variables
sequences = []
journey_targets = []

# Loop over the unique journey IDs
for journey_id in unique_journey_ids:
    # Get the data for the current journey
    journey_data = df[df['journey'] == journey_id].sort_values(by='JourneyStep')
    
    # Get the target variable for the current journey
    journey_target = 1 if 1 in journey_data['sale'].values else 0
    
    # Get the predictor variables for the current journey
    journey_predictors = journey_data.drop(['journey', 'JourneyStep', 'sale'], axis=1).values
    
    # Pad the sequences to have the same length
    journey_predictors = pad_sequences([journey_predictors], maxlen=20, padding='post')[0]
    
    # Add the current journey to the list of sequences and target variables
    sequences.append(journey_predictors)
    journey_targets.append(journey_target)



# Convert the lists to arrays
sequences = np.array(sequences)
journey_targets = np.array(journey_targets)
journey_targets = journey_targets.reshape(-1, 1)

In [ ]:
# Split the data into train, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(sequences, journey_targets, test_size=0.2,shuffle=True, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25,shuffle=True, random_state=0)

# Define the model
model = Sequential()
model.add(Masking(mask_value=0., input_shape=( 20, 36 )))
model.add(LSTM(units=64, return_sequences=False))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', patience=5, mode='min', restore_best_weights=True)

history = model.fit(X_train, y_train, batch_size=512, epochs=100, validation_data=(X_val, y_val), callbacks=[early_stop])

#Save the model for later use
model.save('model.h5')

In [ ]:
# Plot training & validation accuracy values (run them all together)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values (run them all together)

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()


#Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print('Test Loss:', test_loss)
print('Test Acc:', test_accuracy)

# Predict the probabilities of target classes
pred_probs = model.predict(X_test)

# Convert probabilities to binary predictions
predictions = (pred_probs > 0.5).astype(int)

# Calculate the confusion matrix
cm = confusion_matrix(y_test, predictions)

# Plot the confusion matrix
sns.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
# Plot the model architecture
plot_model(model, show_shapes=True, show_layer_names=True)

#Counterfunctional Analysis

y_pred = model.predict(X_test) # predict the class labels
not_converted = [index for index, value in enumerate(y_pred) if value[0] < 0.5] # indices of samples where the model predicts not to convert
X_not_converted = X_test[not_converted]
y_not_converted = y_test[not_converted]
X_whatif= X_not_converted.copy()

# Fuction to find the last step
def get_last_step(sequence):
  for i in range(len(sequence) - 1, -1, -1):
    if sequence[i][29:32].any():
      return i
  return 0

#Change the channel of last step to email firm initiated
for i in range(len(X_whatif)):
 last_step = get_last_step(X_whatif[i])
 for j in range(last_step+1):
  X_whatif[i][j][0:28] = 0
  X_whatif[i][j][28] = 1
    
# Store predictions before and after modification
y_pred_before =model.predict (X_not_converted)
y_pred_before = (y_pred_before > 0.5).astype(int)
y_pred_after = model.predict(X_whatif)
y_pred_after = (y_pred_after > 0.5).astype(int)
# Count the number of cases that have changed predictions
count = 0
for i in range(len(y_pred_before)):
    if y_pred_before[i] != y_pred_after[i]:
        count += 1

# Print the number of cases that have changed predictions
print("Percentage  of cases that have changed class:", (count/9929)*100,"%")
##Percentage  of cases that have changed class: 1.7121563097995771 %##

#Change the first step to email firm initiated
for i in range(len(X_whatif)):
    X_whatif[i][0][0:28] = 0
    X_whatif[i][0][3] = 1
    
# Store predictions before and after modification
y_pred_before =model.predict (X_not_converted)
y_pred_before = (y_pred_before > 0.5).astype(int)
y_pred_after = model.predict(X_whatif)
y_pred_after = (y_pred_after > 0.5).astype(int)
# Count the number of cases that have changed predictions
count = 0
for i in range(len(y_pred_before)):
    if y_pred_before[i] != y_pred_after[i]:
        count += 1

# Print the number of cases that have changed predictions
print("Percentage  of cases that have changed class:", (count/9929)*100,"%")
##Percentage  of cases that have changed class: 4.2199617282707225 %

#Change the first step to directmail 
for i in range(len(X_whatif)):
    X_whatif[i][0][0:28] = 0
    X_whatif[i][0][14] = 1
    
# Store predictions before and after modification
y_pred_before =model.predict (X_not_converted)
y_pred_before = (y_pred_before > 0.5).astype(int)
y_pred_after = model.predict(X_whatif)
y_pred_after = (y_pred_after > 0.5).astype(int)
# Count the number of cases that have changed predictions
count = 0
for i in range(len(y_pred_before)):
    if y_pred_before[i] != y_pred_after[i]:
        count += 1

# Print the number of cases that have changed predictions
print("Percentage  of cases that have changed class:", (count/9929)*100,"%")
##Percentage  of cases that have changed class: 1.3898680632490683  %


#Change the first step to directmail 
for i in range(len(X_whatif)):
    X_whatif[i][0][0:28] = 0
    X_whatif[i][0][28] = 1
    
# Store predictions before and after modification
y_pred_before =model.predict (X_not_converted)
y_pred_before = (y_pred_before > 0.5).astype(int)
y_pred_after = model.predict(X_whatif)
y_pred_after = (y_pred_after > 0.5).astype(int)
# Count the number of cases that have changed predictions
count = 0
for i in range(len(y_pred_before)):
    if y_pred_before[i] != y_pred_after[i]:
        count += 1

# Print the number of cases that have changed predictions
print("Percentage  of cases that have changed class:", (count/9929)*100,"%")
